In [94]:
from os import listdir
from os.path import join, dirname, abspath, isfile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Flatten
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import tensorflow
import glob
from keras.models import load_model
image_size = 260

def train_classifier(img_dir, epochs, batch_size, num_classes = 2, fast_train=False):                                                 # train_function
    print("training")
    
    # model definition
    base_model = tensorflow.keras.applications.EfficientNetB2(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3), classes=2)
    base_model.trainable = True
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    #model.add(Flatten())
    #model.add(BatchNormalization())
    #model.add(Dense(256, activation='relu'))
    #model.add(Dropout(0.4))
    
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.4))
    
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=["accuracy"])
    model.summary()                                                 
    
    # you can set fast_train to true, to check the train function
    if fast_train:
        epochs=1
        batch_size=1
    callbacks = [
        ReduceLROnPlateau(factor=0.4, patience=3, min_lr=0.000001, verbose=1),
        ModelCheckpoint('final_model.hdf5', verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')
    ]
    
    # datagenerator definition
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=30,
        zoom_range=0.1,
        horizontal_flip=True,
        validation_split=0.2)

    train_generator = train_datagen.flow_from_directory(
        img_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='binary',
        subset='training') # set as training data

    validation_generator = train_datagen.flow_from_directory(
        img_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='binary',
        subset='validation') # set as validation data
    
    # fit model
    model.fit_generator(
        train_generator,
        steps_per_epoch = 128,
        validation_data = validation_generator, 
        validation_steps = 64,
        epochs = epochs, callbacks=callbacks)
    
    return model

def classify(path_to_model, test_img_dir):
    x=[]
    result = {}
    model = load_model(path_to_model)
    cnt = 0
    onlyfiles = [f for f in listdir(test_img_dir) if isfile(join(test_img_dir, f))]
    for fname in onlyfiles:
        img = image.load_img(join(test_img_dir,fname), target_size=(image_size, image_size))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        img = np.expand_dims(img, 0)
        cnt += 1
        y_prob = model.predict(img, batch_size=1)
        if y_prob[0][0] >= 0.5:
            result[fname] = 'male'
        else:
            result[fname] = 'female'
        if cnt == 213:
            break
    
    with open('./process_results.json', 'w') as fp:
        json.dump(result, fp)


In [2]:
train_classifier('../input/male-female-final/to_download', 15, 64)

training
31793152/31790344 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb2 (Functional)  (None, 9, 9, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1408)              5632      
_________________________________________________________________
dense (Dense)                (None, 128)               180352    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 7,954,682
Trainable params: 7,884,291
Non-trainable params: 70,391
_________________________________________________________________
Found 80002 images belonging to 2

In [95]:
classify('./final_model.hdf5', '../input/male-female-final/to_download/female')

In [96]:
with open('./process_results.json', 'r') as fp:
    data = json.load(fp)

In [97]:
len(data)

213

In [98]:
data

TypeError: unhashable type: 'slice'